In [14]:
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten, Masking
from keras.layers import LSTM

from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

In [15]:
import pose
from pose_parser import parse_file, detect_perspective
import numpy as np

In [16]:
good_videos = [parse_file("dataset/bicep/bicep_good_"+ str(i) +".npy") for i in range(1,7)]
bad_videos = [parse_file("dataset/bicep/bicep_bad_" + str(i) + ".npy") for i in range(1,11)]

Data shape:  (114, 18, 3)
Mean torso:  360.11485467535044
Data shape:  (101, 18, 3)
Mean torso:  361.91618075058716
Data shape:  (140, 18, 3)
Mean torso:  274.49230143795376
Data shape:  (122, 18, 3)
Mean torso:  276.056770970454
Data shape:  (113, 18, 3)
Mean torso:  386.10674117118276
Data shape:  (117, 18, 3)
Mean torso:  388.8324699035529
Data shape:  (108, 18, 3)
Mean torso:  359.69948300318237
Data shape:  (146, 18, 3)
Mean torso:  362.94356251552193
Data shape:  (92, 18, 3)
Mean torso:  364.56482601788053
Data shape:  (120, 18, 3)
Mean torso:  364.55769028003914
Data shape:  (123, 18, 3)
Mean torso:  271.04817985102164
Data shape:  (100, 18, 3)
Mean torso:  274.544553716319
Data shape:  (127, 18, 3)
Mean torso:  276.5870125620258
Data shape:  (102, 18, 3)
Mean torso:  389.4333451516014
Data shape:  (108, 18, 3)
Mean torso:  366.72702499907354
Data shape:  (80, 18, 3)
Mean torso:  382.9545547331215


In [29]:
xs = []
ys = []
for video in good_videos + bad_videos:
    frames = video

    side = detect_perspective(frames)

    #Angles to calculate
    upperarm_forearm_angles = []
    upperarm_torso_angles = []
    sequence = []

    for frame in frames:
        if (side == pose.Side.right):
            upperarm = pose.Part(frame.relbow, frame.rshoulder)
            forearm = pose.Part(frame.relbow, frame.rwrist)
            torso = pose.Part(frame.rhip, frame.neck)
        else:
            upperarm = pose.Part(frame.lelbow, frame.lshoulder)
            forearm = pose.Part(frame.lelbow, frame.lwrist)
            torso = pose.Part(frame.lhip, frame.neck)
        angle1 = upperarm.calculate_angle(forearm)
        angle2 = upperarm.calculate_angle(torso)
        sequence.append(np.array([angle1, angle2]))
        upperarm_forearm_angles.append(angle1)
        upperarm_torso_angles.append(angle2)
    xs.append(np.array(sequence))
    if (video in good_videos):
        ys.append([0])
    else:
        ys.append([1])

Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Left
Primary arm: Left
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Right
Primary arm: Left
Primary arm: Left
Primary arm: Right


In [30]:
print(len(xs), len(ys))
print(ys)

16 16
[[0], [0], [0], [0], [0], [0], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1]]


In [31]:
#Convert to np array
xs = np.array(xs)
ys = np.array(ys)
print(xs.shape, ys.shape)
#Add padding
from keras.preprocessing import sequence
xs=sequence.pad_sequences(xs, padding='post', dtype='float', truncating='post')
ys=sequence.pad_sequences(ys, padding='post', dtype='float', truncating='post')
seq_len = xs.shape[1]
print(xs.shape, ys.shape)


(16,) (16, 1)
(16, 146, 2) (16, 1)


In [8]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(xs, ys, test_size=0.20, random_state=42)
X_train, y_train = shuffle(X_train, y_train)
X_test, y_test = shuffle(X_test, y_test)


In [10]:
model = Sequential()
model.add(LSTM(256, input_shape=(seq_len, 2)))
model.add(Dense(1, activation='sigmoid'))

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, None, 2)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 16)                1216      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 1,233
Trainable params: 1,233
Non-trainable params: 0
_________________________________________________________________


In [13]:
adam = Adam(lr=0.001)
# chk = ModelCheckpoint('best_model.pkl', monitor='val_acc', save_best_only=True, mode='max', verbose=1)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(x_train, y_train, epochs=100, batch_size=16, validation_data=(x_val,y_val))
# model.predict(np.random.rand(10, 5, n_feats))   # This works

Instructions for updating:
Use tf.cast instead.
Train on 14 samples, validate on 2 samples
Epoch 1/100
14/14 [==============================] - 1s - loss: 0.7572 - acc: 0.5714 - val_loss: 0.7197 - val_acc: 0.5000
Epoch 2/100
14/14 [==============================] - 0s - loss: 0.7478 - acc: 0.5714 - val_loss: 0.7190 - val_acc: 0.5000
Epoch 3/100
14/14 [==============================] - 0s - loss: 0.7384 - acc: 0.5714 - val_loss: 0.7184 - val_acc: 0.5000
Epoch 4/100
14/14 [==============================] - 0s - loss: 0.7290 - acc: 0.5714 - val_loss: 0.7181 - val_acc: 0.5000
Epoch 5/100
14/14 [==============================] - 0s - loss: 0.7188 - acc: 0.5000 - val_loss: 0.7182 - val_acc: 0.5000
Epoch 6/100
14/14 [==============================] - 0s - loss: 0.7104 - acc: 0.5000 - val_loss: 0.7194 - val_acc: 0.5000
Epoch 7/100
14/14 [==============================] - 0s - loss: 0.7057 - acc: 0.4286 - val_loss: 0.7225 - val_acc: 0.5000
Epoch 8/100
14/14 [==============================] - 0s